# The problem: inferring grids

When we acquire data we often do so point-by-point, or chunk-by-chunk. And in general it is not possible to know in advance what shape exactly the final data will have. For multi-dimensional data this means that we don't always know on what kind of of grid the data lies, if any. That information, however, is important for a variety of tasks we would like to perform, such as slicing our data, or plotting projections of it. And we want to do all of these already when we don't have the full data set yet, i.e., while a data acquisition is still running.

Things would of course be much easier if a grid of the right shape was pre-allocated and then gradually filled. But most data saving is not done that way (like in qcodes, for example). For that reason, we look at a few ways on how to infer grids from tabular data, where data is saved row-by-row.

# Setting up

These are the important imports and some tool. Execute this first.

In [1]:
%matplotlib widget

In [2]:
import numpy as np
from matplotlib import pyplot as plt
plt.close('all')

from plottr.data import datadict as dd
from plottr.utils import num
from plottr.utils.num import _find_switches, is_invalid
from plottr.plot.mpl import ppcolormesh_from_meshgrid


def plot_image(x, y, z, ax=None, title=''):
    """Plot a grid as image. The arrays x, y, z need to be in meshgrid form."""
    
    if ax is None:
        fig, ax = plt.subplots(1, 1)
        fig.canvas.layout.width = '500px'
        fig.canvas.layout.height = '500px'
        fig.subplots_adjust(top=0.9)
        fig.suptitle(title)
        
    _x = x.flatten()
    _y = y.flatten()
    x0, x1 = _x[~np.isnan(_x)].min(), _x[~np.isnan(_x)].max()
    y0, y1 = _y[~np.isnan(_y)].min(), _y[~np.isnan(_y)].max()
    extent = [x0, x1, y0, y1]
    z2 = z.copy()
    z2 = z2 if x[0, 0] < x[1, 0] else z2[::-1, :]
    z2 = z2 if y[0, 0] < y[0, 1] else z2[:, ::-1]
    ax.imshow(z2.T, origin='lower', extent=extent, aspect='auto')

    
def plot_grid2d(x, y, z, title=''):
    """Plot a grid as image and pcolormesh side by side. x, y, z need to be meshgrids."""
    
    fig, axes = plt.subplots(1, 2, sharex='all', sharey='all')
    ax = axes[0]
    plot_image(x, y, z, ax=ax)
    
    ax = axes[1]
    ppcolormesh_from_meshgrid(ax, x, y, z)
    
    fig.tight_layout()
    fig.canvas.layout.width = '800px'
    fig.canvas.layout.height = '400px'
    fig.suptitle(title)
    fig.subplots_adjust(top=0.9)
    

def add_noise(grid2d, scale='auto'):
    if scale == 'auto':
        scale = grid2d.std() * 0.2
        
    for irow, row in enumerate(grid2d):
        for ipt, pt in enumerate(row):
            grid2d[irow, ipt] += np.random.normal(scale=scale)
    
    return grid2d
    

def gridpattern(x, y, noise=False, noise_scale='auto'):
    xx, yy = np.meshgrid(x, y, indexing='ij')
    
    if noise:
        xx = add_noise(xx, scale=noise_scale)
        yy = add_noise(yy, scale=noise_scale)

    zz = np.sinc((xx**2 + yy**2)**.5)
    for ix, _x in enumerate(x):
        for iy, _y in enumerate(y):
            if (ix%2 and iy%2) or (not ix%2 and not iy%2):
                zz[ix, iy] -= -0.1
    
    return xx, yy, zz


def find_and_plot_switches(**arrs):
    fig, axes = plt.subplots(len(arrs), 1, sharex='all')
    if len(arrs) == 1:
        axes = [axes]
    
    i = 0
    for k, a in arrs.items():
        switches = _find_switches(a)
        axes[i].plot(a, drawstyle='steps-mid', color='b')
        for s in switches:
            axes[i].axvline(s, color='r')
        axes[i].set_ylabel(k)
        i+=1
    axes[-1].set_xlabel('index')
    
    return switches, axes

# Inferring grids from sweep directions

The main method we'll be using to infer the grid is to look at systematics in the coordinates (the *independents*) of the data.
Since our main focus is to look at measurements, we look at the way the coordinates are swept or rastered -- this is by far the most common way how control parameters are changed in the lab.

Very commonly, we sweep over our coordinates in nested loops, which then naturally form a grid. Coordinates then typically look something like this:

In [3]:
# define two coordinate axes
x = np.linspace(-3, 3, 21)
y = np.linspace(-2, 2, 15)

# make some fake data on a grid spanned by x and y
# internally, we use np.meshgrid(x, y, indexing='ij'), which produces a grid 
# as if we had looped over x and y, x being the outer loop.
xx, yy, zz = gridpattern(x, y)

# print the shapes
print(xx.shape, yy.shape, zz.shape)

# plot the grid as image
plot_image(xx, yy, zz)

(21, 15) (21, 15) (21, 15)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

When setting this up, we of course know exactly the shape, and we can do all operations like slicing, plotting, etc. right away, as shown above.
(**Note**: to visualize the grid, the data is overlayed with a checker board pattern)

However, we might not have that information in the final data (or it could be that we didn't finish the sweep, and we only have parts of the grid). The only thing we can rely on in the end, is the data. And if it's stored in a tabular format, the shape information may be gone or incorrect.

We will start with flattened data, like:

In [4]:
x1d = xx.flatten()
y1d = yy.flatten()
z1d = zz.flatten()

To reconstruct the grid, we can analyze the values of the coordinates that occur in the data. In general that can be tricky -- you would need to look at all occuring values and then sort the data accordingly onto the grid formed by all coordinates found.

However, for the cases where grids are useful to start with, the experimenter will (hopefully!) have systematically swept over the coordinates (as we have done above, essentially).
If the sweeps are monotonous, we can reconstruct grids simply using `np.reshape`. The only thing we need to figure out is the shape of the grid we want to make.

To do that, we can simply look at the evolution of the coordinates:

In [5]:
find_and_plot_switches(x=x1d, y=y1d)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([ 14,  29,  44,  59,  74,  89, 104, 119, 134, 149, 164, 179, 194,
        209, 224, 239, 254, 269, 284, 299]),
 array([<matplotlib.axes._subplots.AxesSubplot object at 0x7f3b02bb6e10>,
       dtype=object))

When we assume that sweep direction is monotonous, then we can simply count the number switches in direction (the period) to figure out how often an axis dimension is swept. The suspected switches are marked in red in the plot above.

From these periods it is then easy to get the shape: Here we see that `y` is repeated 21 times -- that means 21 is the number of `x` values on the grid, and the total size divided by 21 is the number of `y` values.

This basic principle is in a function that guesses the grid shape -- `plottr.utils.num.guess_grid_from_sweep_direction` --, which is automatically used in `plottr.data.datadict.datadict_to_meshgrid`.

In [6]:
order, shape = num.guess_grid_from_sweep_direction(x=x1d, y=y1d)
print(order, shape)

['x', 'y'] (21, 15)


In [7]:
plot_image(x1d.reshape(shape), y1d.reshape(shape), z1d.reshape(shape))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

It's important to note that order of course matters. Consider this example:

In [8]:
order, shape = num.guess_grid_from_sweep_direction(not_really_x=y1d, not_really_y=x1d)
print(order, shape)

['not_really_y', 'not_really_x'] (21, 15)


The function that determines the grid shape gives us the correct answer -- but now the roles of x and y are swapped, because `not_really_y` is now the outer loop. This is important to keep in mind when doing things programatically.

# Irregular grids

You might have wondered why were looking at sweep patterns rather than unique values, which might be easier to analyze.
The reason is that it's entirely possible to have a well-defined grid even when the coordinates in each row/column are not repeating exactly.

## Noise

One example is when the coordinate itself is subject to noise or other variations:

In [9]:
x = np.linspace(0, 2, 5)
y = np.linspace(-2, 2, 9)

# steps are 0.5 on each coordinate -- add some noise on a scale that's somewhat smaller
xx, yy, zz = gridpattern(x, y, noise=True, noise_scale=0.2)

# the data we'll get in practice is flattened again
x1d = xx.flatten()
y1d = yy.flatten()
z1d = zz.flatten()

# plot coordinatates
find_and_plot_switches(x=x1d, y=y1d)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([ 8, 17, 26, 35]),
 array([<matplotlib.axes._subplots.AxesSubplot object at 0x7f3b02a69d10>,
       dtype=object))

It's obvious that this data would be hard to sort back onto a grid by looking at actual values. But because the noise is less than the (intentional) variation between the coordinates, we can still infer the grid shape by identifying large switches:

In [10]:
order, shape = num.guess_grid_from_sweep_direction(x=x1d, y=y1d)
print(order, shape)

plot_grid2d(x1d.reshape(shape), y1d.reshape(shape), z1d.reshape(shape))

['x', 'y'] (5, 9)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

The left plot shows the grid plotted as image, whereas a the right is showing a more accurate representation where the coordinates are moved by the added noise.

## Adaptive measurements

Another example where irregular grids can occur is an adaptive sweep, where the coordinates in one dimension depend on the values of another. 
A simple, artificial example, we again look at an image of the grid and also the 'real' representation.

In [11]:
x = np.linspace(-2, 2, 11)
y = np.linspace(-1, 1, 11)
xx, yy = np.meshgrid(x, y, indexing='ij')

# we're stretching the grid a bit, depending on the value of x
for i in range(y.size):
    yy[i,:] *= (2 * np.exp(-x[i]**2/2.))

# mock data: a gaussian peak in 2D
zz = np.exp(-xx**2 - yy**2)

plot_grid2d(xx, yy, zz)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

But of course, looking at switches still works. 

**Note:** When the distortion gets very bad, then it can become difficult to detect switches (when the magnitude of a switch is not much larger than the variations in the coordinate sweep). Then our method can fail.

In [12]:
x1d = xx.flatten()
y1d = yy.flatten()
z1d = zz.flatten()
find_and_plot_switches(x=x1d, y=y1d)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([ 10,  21,  32,  43,  54,  65,  76,  87,  98, 109]),
 array([<matplotlib.axes._subplots.AxesSubplot object at 0x7f3b0288f110>,
       dtype=object))

In [13]:
order, shape = num.guess_grid_from_sweep_direction(x=x1d, y=y1d)
print(order, shape)

plot_grid2d(x1d.reshape(shape), y1d.reshape(shape), z1d.reshape(shape))

['x', 'y'] (11, 11)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Incomplete data

An important task is gridding of data where the target grid hasn't been fully filled.
This arises, for example, when a measurement is still ongoing, or has been aborted before finishing.
Then, the size of the data is generally not readily suited for reshaping.

In this case we have implemented functionality to 'pad' the data such that a grid is possible again. 
To do that we find the smallest possible grid that encloses the data, fill the data with NaN, and then reshape.
To make our life a bit easier, we use the DataDict format which has tools for this.

In [14]:
# define two coordinate axes
x = np.linspace(-3, 3, 21)
y = np.linspace(-2, 2, 15)

# make some fake data on a grid spanned by x and y
xx, yy, zz = gridpattern(x, y)

# print the full shapes
print("Full shapes:", xx.shape, yy.shape, zz.shape)

# now make flattened data where some entries are missing at the end
nmissing = 100
x1d = xx.flatten()[:-nmissing]
y1d = yy.flatten()[:-nmissing]
z1d = zz.flatten()[:-nmissing]

# note: we can still find the grid!
order, shape = num.guess_grid_from_sweep_direction(x=x1d, y=y1d)
print("Grid shape of the incomplete data:", shape)

# reconstruct the correct grid
# to do so we use the datadict format and its convenience tools:
data1d = dd.DataDict(
    x = dict(values=x1d),
    y = dict(values=y1d),
    z = dict(values=z1d, axes=['x', 'y'])
)

# guessing the grid, padding, and reshaping is all automatic here
data2d = dd.datadict_to_meshgrid(data1d)
print("DataDict shape:", data2d.shape())

# plot the grid
plot_image(data2d.data_vals('x'), data2d.data_vals('y'), data2d.data_vals('z'))

Full shapes: (21, 15) (21, 15) (21, 15)
Grid shape of the incomplete data: (15, 15)
DataDict shape: (15, 15)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …